In [2]:
%reload_ext autoreload
%autoreload 2

import os
import xarray as xr
import numpy as np
import sys
import pandas as pd

sys.path.append("..")
import utils
import plot_utils

In [4]:
fold = f"../../results/SST/CMIP6/"
dir_dic = utils.dirtodict(fold)
models = sorted(dir_dic.keys())[1:]
experiments = ["ssp126", "ssp585"]
# ranges = [(2010, 2019), (2030, 2039), (2050, 2059), (2070, 2079)]
ranges = [(i, i + 19) for i in range(2000, 2100, 20)]

fold_ = "../../data/CMIP6/"
dir_dic_ = utils.dirtodict(fold_)


# for model in models:
#     variants = sorted(list(dir_dic[model].keys()))[1:]
#     for i, variant in enumerate(variants):


n_var_res = {
    model: {"# var": len(sorted(list(dir_dic[model].keys()))[1:]), "Res.": None}
    for model in models
}

trend_mean = {
    model: {
        experiment: {
            range_: {
                "Trend": None,
                "p": None,
                "SST": None,
            }
            for range_ in ranges
        }
        for experiment in experiments
    }
    for model in models
}

ensemble_median = {
    model: {experiment: None for experiment in experiments} for model in models
}

for model in models[:]:
    variants = sorted(list(dir_dic[model].keys()))[1:]
    n_vars = len(variants)
    median_s1 = np.zeros((100, n_vars))
    median_s5 = np.zeros((100, n_vars))
    for i, variant in enumerate(variants):
        files = dir_dic[model][variant][".files"]
        path_hist = [f for f in files if "historical" in f][0]
        path_s1 = [f for f in files if "ssp126" in f][0]
        path_s5 = [f for f in files if "ssp585" in f][0]
        ds_hist = xr.open_dataset(path_hist)
        median_s1[:, i] = (
            xr.concat((ds_hist, xr.open_dataset(path_s1)), dim="year")
            .sel(year=slice(2000, 2099))
            .tos.values
        )
        median_s5[:, i] = (
            xr.concat((ds_hist, xr.open_dataset(path_s5)), dim="year")
            .sel(year=slice(2000, 2099))
            .tos.values
        )
        if i == 0:
            path = dir_dic_[model][variant]["tos"]["historical"][".files"][0]
            res = xr.open_dataset(path).attrs["nominal_resolution"]
            n_var_res[model]["Res."] = res

    ensemble_median[model]["ssp126"] = np.median(median_s1, axis=1)
    ensemble_median[model]["ssp585"] = np.median(median_s5, axis=1)

    for experiment in ["ssp126", "ssp585"]:
        for range_ in ranges:
            (
                trend_mean[model][experiment][range_]["Trend"],
                trend_mean[model][experiment][range_]["p"],
            ) = (
                utils.trend(
                    ensemble_median[model][experiment][
                        range_[0] - 2000 : range_[1] - 2000 + 1
                    ]
                )
                
            )
            trend_mean[model][experiment][range_]["SST"] = np.mean(
                ensemble_median[model][experiment][
                    range_[0] - 2000 : range_[1] - 2000 + 1
                ]
            )

/home/prossello/MHW_article/myenv/lib/python3.9/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/home/prossello/MHW_article/myenv/lib/python3.9/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/home/prossello/MHW_article/myenv/lib/python3.9/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/home/prossello/MHW_article/myenv/lib/python3.9/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/home/prossello/MHW_article/myenv/lib/python3.9/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value enco

In [5]:
ranges_str = [f"{range_[0]}-{range_[1]}" for range_ in ranges]
ranges_str = [str(range_[0])[-2:] + '-' + str(range_[1])[-2:]  for range_ in ranges]

simple_columns = ["Res.", "# var."]

# Create a list of tuples for the simple column headers
simple_column_tuples = [(col, '') for col in simple_columns]

# Create a list of tuples for the MultiIndex column headers
multiindex_column_tuples = [(range_str, sub_col) for range_str in ranges_str for sub_col in ["SST", "Trend"]]

# Combine the simple and MultiIndex column headers
all_column_tuples = simple_column_tuples + multiindex_column_tuples

# Create a MultiIndex from the combined column headers
columns = pd.MultiIndex.from_tuples(all_column_tuples)

# Create the DataFrame with the desired column headers
df_s1 = pd.DataFrame(index=models, columns=columns)
df_s5 = pd.DataFrame(index=models, columns=columns)

dfs = {'ssp126': df_s1, 'ssp585': df_s5}

add = ''
for thing in ['SST', 'Trend']:
    for experiment in experiments:
        for model in models:
            for i, range_ in enumerate(ranges_str):
                add = ''
                if thing=='Trend':
                    if trend_mean[model][experiment][ranges[i]]['p']<0.05:
                        add = '*'
                dfs[experiment].loc[model, (range_, thing)] = str(round(trend_mean[model][experiment][ranges[i]][thing],2))+add

            

In [6]:
for experiment in experiments:
    dfs[experiment].loc[:, ('# var.')] = [f"{n_var_res[model]['# var']}" for model in models]
    dfs[experiment].loc[:, ('Res.')] = [f"{n_var_res[model]['Res.'][:-3]}" for model in models]


In [7]:
dfs['ssp126']

Res. # var.  00-19         20-39         40-59          \
                                SST  Trend    SST  Trend    SST   Trend   
ACCESS-CM2        250      5  20.15  0.02*   21.0  0.03*  21.52   0.03*   
ACCESS-ESM1-5     250      5  20.53  0.04*  21.33  0.04*  21.94   0.01*   
CMCC-CM2-SR5      100      1  20.73   0.01  21.09   0.02  21.84   0.06*   
CMCC-ESM2         100      1  20.32    0.0  20.61  0.07*  21.57   0.06*   
CNRM-CM6-1        100      5  20.07  0.03*  20.62  0.02*  20.84    0.01   
CNRM-CM6-1-HR      25      1  19.09  0.03*  19.84  0.03*  20.35   0.02*   
CNRM-ESM2-1       100      3  20.53  0.02*  21.04  0.02*  21.29    -0.0   
CanESM5           100      4  19.43  0.04*  20.21  0.03*  20.71    0.01   
EC-Earth3         100      1  20.15  0.04*  20.82   0.02  21.25  -0.03*   
EC-Earth3-Veg     100      1  20.35  0.05*  20.98   0.02  20.88    0.01   
EC-Earth3-Veg-LR  100      1  19.87  0.03*  20.42   0.01  20.77   0.03*   
GFDL-ESM4          50      1  19.88  0.02*  20.45  -0.01   20.5   0.02*   
HadGEM3-GC31-LL   100      1  20.19  0.04*  20.89  0.02*  21.33   0.02*   
HadGEM3-GC31-MM    25      1   20.0   0.01  20.48  0.07*  21.11     0.0   
IPSL-CM6A-LR      100      5  19.29  0.02*  19.78  0.03*  20.22   0.02*   
MIROC6            100      3  21.42  0.02*  21.85  0.03*  22.26   0.02*   
MPI-ESM1-2-HR      50      2  19.78  0.02*  20.15  0.02*  20.33     0.0   
MPI-ESM1-2-LR     250      5  19.06   0.01  19.32  0.02*  19.49   0.02*   
MRI-ESM2-0        100      5  19.17  0.02*  19.82  0.02*  19.94   -0.01   
NorESM2-LM        100      1  20.92  0.05*  21.45  0.04*  21.54    -0.0   
NorESM2-MM        100      1  20.55   0.02  21.07  -0.02  21.31     0.0   
UKESM1-0-LL       100      5  20.17  0.04*  21.02  0.04*  21.56   0.03*   

                  60-79          80-99          
                    SST   Trend    SST   Trend  
ACCESS-CM2        21.78    -0.0  21.86   -0.01  
ACCESS-ESM1-5     22.05   -0.01   22.1    0.01  
CMCC-CM2-SR5      22.37   -0.01  22.61   0.03*  
CMCC-ESM2         22.16    0.03  22.54   0.03*  
CNRM-CM6-1        20.82    0.01  21.08   -0.01  
CNRM-CM6-1-HR     20.41   -0.01  20.43    0.01  
CNRM-ESM2-1       21.47     0.0  21.46    0.01  
CanESM5           20.91     0.0  20.82  -0.02*  
EC-Earth3         21.31     0.0  21.09   -0.02  
EC-Earth3-Veg     21.08    0.01  21.43   0.03*  
EC-Earth3-Veg-LR  20.72   -0.01  20.72    0.02  
GFDL-ESM4         20.64   -0.01  20.39   -0.01  
HadGEM3-GC31-LL   21.66    0.01  21.84   0.03*  
HadGEM3-GC31-MM   21.22   0.03*  21.45  -0.03*  
IPSL-CM6A-LR      20.36    0.01  20.41    -0.0  
MIROC6            22.44    0.01  22.19  -0.02*  
MPI-ESM1-2-HR     20.26  -0.02*  20.18   -0.01  
MPI-ESM1-2-LR     19.63    -0.0  19.46     0.0  
MRI-ESM2-0        19.84  -0.01*  19.67  -0.01*  
NorESM2-LM        21.76    0.01   21.6    0.01  
NorESM2-MM        21.56     0.0   21.6   -0.02  
UKESM1-0-LL       21.89    0.01  22.09   0.01*

In [72]:
for experiment in experiments:

    dfs[experiment].to_latex(f'../../tables/table_2_{experiment}.tex',
                        bold_rows=True,
                        header=True,
                        column_format='|c|c|c|c|c|c|c|c|c|c|c|c|c|c|',
                        caption="")

/tmp/ipykernel_2294797/1052186499.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  dfs['ssp126'].to_latex('../../tables/table_2_ssp126.tex',
